1. Get the Slack export data by following [these instructions](https://slack.com/intl/en-ca/help/articles/201658943-Export-your-workspace-data#free-and-standard-plans-1)
2. Set up the [Python Google Docs API](https://developers.google.com/docs/api/quickstart/python)
3. Run the code below

In [27]:
import os

if not os.path.isfile("credentials.json"):
    raise FileExistsError("credentials.json must be in this working directory")

In [4]:
import glob
import zipfile


data_dir = "out"

# Unzip all exported Slack messages into out/
for file_name in glob.glob('*.zip'):
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

In [74]:
drive_dir = "11DLP8O_cb-2PSR6hBGA0oXlAY6GVh6Lx"  # This is the "Slack Exports" directory

Set up credentials for GDrive and GCloud

In [56]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

drive_service = build('drive', 'v3', credentials=creds)
docs_service = build('docs', 'v1', credentials=creds)

here
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=886406488177-meq32e40sgl8fvvkhbqdtpemq9bctn1d.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54137%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=0NJy2IA23TTlSAsFuXMmwCXZwntegn&access_type=offline


In [72]:
def create_file(title):
    body = {
        'title': title,
    }
    return docs_service.documents().create(body=body).execute()

def create_dir(name):
    file_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    return drive_service.files().create(body=file_metadata,
                                        fields='id').execute()

def move(file_id, new_folder_id):
    # Retrieve the existing parents to remove
    file = drive_service.files().get(fileId=file_id,
                                     fields='parents').execute()
    previous_parents = ",".join(file.get('parents'))
    # Move the file to the new folder
    return drive_service.files().update(fileId=file_id,
                                        addParents=new_folder_id,
                                        removeParents=previous_parents,
                                        fields='id, parents').execute()

def addText(file_id, text):
    requests = [
         {
            'insertText': {
                'location': {
                    'index': 1,
                },
                'text': text
            }
        }
    ]

    return docs_service.documents().batchUpdate(
        documentId=file_id, body={'requests': requests}).execute()

In [98]:
from datetime import datetime

def get_date(ts):
    return datetime.utcfromtimestamp(int(float(ts))).strftime('%Y-%m-%d %H:%M:%S')

def format_messages(messages):
    texts = []
    
    for msg in messages:
        if msg.get("type", None) != "message":
            continue
        user = msg.get("user_profile", {})
        name = user.get("display_name", None)
        ts = msg.get("ts", None)
        
        if not name or not ts:
            continue
        
        text = msg.get("text", "")
        texts.append(f"{get_date(ts)} - {name}: {text}")
    
    return texts

In [ ]:
done = set()

In [ ]:
import json

for channel_name in os.listdir(data_dir):
    if channel_name in done:
        continue
    
    print(f"Creating {channel_name}")
    
    if os.path.isdir(os.path.join(data_dir, channel_name)):
        all_texts = []
        
        for date_name in os.listdir(os.path.join(data_dir, channel_name)):
            file_path = os.path.join(data_dir, channel_name, date_name)
            
            with open(file_path) as f:
                messages = json.load(f)
                all_texts = all_texts + format_messages(messages)
    
    file = create_file(channel_name)
    all_texts = "\n".join(all_texts)
    if all_texts:
        addText(file["documentId"], all_texts)
    move(file["documentId"], drive_dir)
    
    done.add(channel_name)
    print("done")

Creating socials
done
Creating emojis
